# MNISTのサンプルをMXNetで試す
http://mxnet.io/tutorials/python/mnist.html

In [1]:
import mxnet as mx
mnist = mx.test_utils.get_mnist()

In [2]:
batch_size = 100
train_iter = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

## Training

In [3]:
data = mx.sym.var('data')
# Flatten the data from 4-D shape into 2-D (batch_size, num_channel*width*height)
data = mx.sym.flatten(data=data)

In [4]:
# The first fully-connected layer and the corresponding activation function
fc1  = mx.sym.FullyConnected(data=data, num_hidden=128)
act1 = mx.sym.Activation(data=fc1, act_type="relu")

# The second fully-connected layer and the corresponding activation function
fc2  = mx.sym.FullyConnected(data=act1, num_hidden = 64)
act2 = mx.sym.Activation(data=fc2, act_type="relu")

In [5]:
# MNIST has 10 classes
fc3  = mx.sym.FullyConnected(data=act2, num_hidden=10)
# Softmax with cross entropy loss
mlp  = mx.sym.SoftmaxOutput(data=fc3, name='softmax')

In [6]:
import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
# create a trainable module on CPU
mlp_model = mx.mod.Module(symbol=mlp, context=mx.cpu())
mlp_model.fit(train_iter,  # train data
              eval_data=val_iter,  # validation data
              optimizer='sgd',  # use SGD to train
              optimizer_params={'learning_rate':0.1},  # use fixed learning rate
              eval_metric='acc',  # report accuracy during training
              batch_end_callback = mx.callback.Speedometer(batch_size, 100), # output progress for each 100 data batches
              num_epoch=10)  # train for at most 10 dataset passes

INFO:root:Epoch[0] Batch [100]	Speed: 54461.22 samples/sec	accuracy=0.111980
INFO:root:Epoch[0] Batch [200]	Speed: 42708.07 samples/sec	accuracy=0.109500
INFO:root:Epoch[0] Batch [300]	Speed: 49011.93 samples/sec	accuracy=0.111900
INFO:root:Epoch[0] Batch [400]	Speed: 49821.40 samples/sec	accuracy=0.114100
INFO:root:Epoch[0] Batch [500]	Speed: 53472.24 samples/sec	accuracy=0.141400
INFO:root:Epoch[0] Train-accuracy=0.299293
INFO:root:Epoch[0] Time cost=1.209
INFO:root:Epoch[0] Validation-accuracy=0.475200
INFO:root:Epoch[1] Batch [100]	Speed: 37295.47 samples/sec	accuracy=0.535149
INFO:root:Epoch[1] Batch [200]	Speed: 43980.76 samples/sec	accuracy=0.697900
INFO:root:Epoch[1] Batch [300]	Speed: 47291.63 samples/sec	accuracy=0.790500
INFO:root:Epoch[1] Batch [400]	Speed: 52344.26 samples/sec	accuracy=0.824500
INFO:root:Epoch[1] Batch [500]	Speed: 54427.44 samples/sec	accuracy=0.834800
INFO:root:Epoch[1] Train-accuracy=0.836263
INFO:root:Epoch[1] Time cost=1.310
INFO:root:Epoch[1] Validat

## 予測

In [7]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = mlp_model.predict(test_iter)
assert prob.shape == (10000, 10)

In [8]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
# predict accuracy of mlp
acc = mx.metric.Accuracy()
mlp_model.score(test_iter, acc)
print(acc)
assert acc.get()[1] > 0.96

EvalMetric: {'accuracy': 0.96879999999999999}


## CNN

In [9]:
data = mx.sym.var('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type="tanh")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type="tanh")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type="tanh")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=10)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

In [10]:
# create a trainable module on GPU 0
lenet_model = mx.mod.Module(symbol=lenet, context=mx.cpu())
# train with the same
lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 100),
                num_epoch=10)

INFO:root:Epoch[0] Batch [100]	Speed: 877.27 samples/sec	accuracy=0.111584
INFO:root:Epoch[0] Batch [200]	Speed: 898.99 samples/sec	accuracy=0.109500
INFO:root:Epoch[0] Batch [300]	Speed: 918.36 samples/sec	accuracy=0.111900
INFO:root:Epoch[0] Batch [400]	Speed: 855.08 samples/sec	accuracy=0.114100
INFO:root:Epoch[0] Batch [500]	Speed: 795.08 samples/sec	accuracy=0.119800
INFO:root:Epoch[0] Train-accuracy=0.105859
INFO:root:Epoch[0] Time cost=68.886
INFO:root:Epoch[0] Validation-accuracy=0.113500
INFO:root:Epoch[1] Batch [100]	Speed: 886.51 samples/sec	accuracy=0.112970
INFO:root:Epoch[1] Batch [200]	Speed: 968.01 samples/sec	accuracy=0.200600
INFO:root:Epoch[1] Batch [300]	Speed: 974.56 samples/sec	accuracy=0.671400
INFO:root:Epoch[1] Batch [400]	Speed: 965.08 samples/sec	accuracy=0.878000
INFO:root:Epoch[1] Batch [500]	Speed: 970.65 samples/sec	accuracy=0.902100
INFO:root:Epoch[1] Train-accuracy=0.920606
INFO:root:Epoch[1] Time cost=62.811
INFO:root:Epoch[1] Validation-accuracy=0.936

### 予測

In [11]:
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = lenet_model.predict(test_iter)
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
# predict accuracy for lenet
acc = mx.metric.Accuracy()
lenet_model.score(test_iter, acc)
print(acc)
assert acc.get()[1] > 0.98

EvalMetric: {'accuracy': 0.98809999999999998}
